In [ ]:
!pip install gymnasium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 KB 41.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import gymnasium as gym

# SCIMAI implementation (our inspiration)

One way to do this would be to include the current storage level as part of the state representation that the agent observes. For example, you might include a variable that tracks the number of units currently stored in the environment.

Next, you would need to modify the reward function of the environment to penalize the agent for exceeding the maximum storage capacity. For example, you might subtract a certain amount from the reward for each additional unit produced or stored beyond the capacity limit.

In [ ]:
class State:
    """
    We choose the state vector to include all current stock levels for each 
    warehouse and product type, plus the last demand values.
    """

    def __init__(self, product_types_num, distr_warehouses_num, T,
                 demand_history, t=0): #T is the number of time steps, t is the current timestep
        '''All factory stocks and distr. warehouses stock are set to 0'''
        self.product_types_num = product_types_num
        self.factory_stocks = np.zeros(
            (self.product_types_num,),
            dtype=np.int32)
        self.distr_warehouses_num = distr_warehouses_num
        self.distr_warehouses_stocks = np.zeros(
            (self.distr_warehouses_num, self.product_types_num),
            dtype=np.int32) # This array represents the stock levels at each distributed warehouse for each product type
        self.T = T
        self.demand_history = demand_history # A list of lists of demand values for each warehouse and product type
        self.t = t

    def to_array(self): 
        '''
        returns a flattened array of the state values. returns a flattened numpy array containing
        the current stock levels for each warehouse and product type, as well as the last demand values, and the current time step.
        '''
        return np.concatenate((
            self.factory_stocks,
            self.distr_warehouses_stocks.flatten(),
            np.hstack(list(chain(*chain(*self.demand_history)))),
            [self.t]))

    def stock_levels(self): #returns an array of the current stock levels for each warehouse and product type
        return np.concatenate((
            self.factory_stocks,
            self.distr_warehouses_stocks.flatten()))

In [ ]:
class Action:
    """
    The action vector consists of production and shipping controls.
    """

    def __init__(self, product_types_num, distr_warehouses_num):
        self.production_level = np.zeros(
            (product_types_num,),
            dtype=np.int32)
        self.shipped_stocks = np.zeros(
            (distr_warehouses_num, product_types_num),
            dtype=np.int32)

In [ ]:
class SupplyChainEnvironment:
    """
    We designed a divergent two-echelon supply chain that includes a single 
    factory, multiple distribution warehouses, and multiple product types over 
    a fixed number of time steps. At each time step, the agent is asked to find 
    the number of products to be produced and preserved at the factory, as well 
    as the number of products to be shipped to different distribution 
    warehouses. To make the supply chain more realistic, we set capacity 
    constraints on warehouses (and consequently, on how many units to produce 
    at the factory), along with storage and transportation costs. 
    """

    def __init__(self):
        # number of product types (e.g., 2 product types)
        self.product_types_num = 2
        # number of distribution warehouses (e.g., 2 distribution warehouses)
        self.distr_warehouses_num = 2
        # final time step (e.g., an episode takes 25 time steps)
        self.T = 25

        # maximum demand value, units (e.g., [3, 6])
        self.d_max = np.array(
            [3, 6],
            np.int32)
        # maximum demand variation according to a uniform distribution,
        # units (e.g., [2, 1])
        self.d_var = np.array(
            [2, 1],
            np.int32)

        # sale prices, per unit (e.g., [20, 10])
        self.sale_prices = np.array(
            [20, 10],
            np.int32)
        
        # production costs, per unit (e.g., [2, 1])
        self.production_costs = np.array(
            [2, 1],
            np.int32)

        # storage capacities for each product type at each warehouse,
        # units (e.g., [[3, 4], [6, 8], [9, 12]])
        self.storage_capacities = np.array(
            [[3, 4], [6, 8], [9, 12]],
            np.int32)

        # storage costs of each product type at each warehouse,
        # per unit (e.g., [[6, 3], [4, 2], [2, 1]])
        self.storage_costs = np.array(
            [[6, 3], [4, 2], [2, 1]],
            np.float32)
        # transportation costs of each product type for each distribution
        # warehouse, per unit (e.g., [[.1, .3], [.2, .6]])
        self.transportation_costs = np.array(
            [[.1, .3], [.2, .6]],
            np.float32)

        # penalty costs, per unit (e.g., [10, 5])
        self.penalty_costs = .5*self.sale_prices
        
        self.reset() # Reset the environment to its initial state at the beginning of a new episode (function defined just below)

    def reset(self, demand_history_len=5):
        # (five) demand values observed
        self.demand_history = collections.deque(maxlen=demand_history_len) # Deque is a data structure that allows for efficient insertion and removal of elements from both ends.

        for d in range(demand_history_len):
            self.demand_history.append(np.zeros(
                (self.distr_warehouses_num, self.product_types_num),
                dtype=np.int32))
        self.t = 0

    def demand(self, j, i, t):
        # we simulate a seasonal behavior by representing the demand as a
        # co-sinusoidal function with a stochastic component (a random variable
        # assumed to be distributed according to a uniform distribution),
        # in order to evaluate the agent
        # This line calculates the demand for product i at warehouse j and time t
        demand = np.round(
            self.d_max[i-1]/2 +
            self.d_max[i-1]/2*np.cos(4*np.pi*(2*j*i+t)/self.T) +
            np.random.randint(0, self.d_var[i-1]+1))
        return demand

    def initial_state(self):
        return State(self.product_types_num, self.distr_warehouses_num,
                     self.T, list(self.demand_history))

    def step(self, state, action):
        # Get the demand of each product for each warehouse
        demands = np.fromfunction(
            lambda j, i: self.demand(j+1, i+1, self.t),
            (self.distr_warehouses_num, self.product_types_num),
            dtype=np.int32)
        # next state
        next_state = State(self.product_types_num, self.distr_warehouses_num,
                           self.T, list(self.demand_history))

        """
        This updates the inventory level of the factory by subtracting the products used for production (action.production_level)
        and adding the products shipped to the warehouses (action.shipped_stocks).
        The resulting inventory level is then capped at the maximum storage capacity of the factory (self.storage_capacities[0]).
        """
        next_state.factory_stocks = np.minimum(
            np.subtract(np.add(state.factory_stocks,
                               action.production_level),
                        np.sum(action.shipped_stocks, axis=0)
                        ),
            self.storage_capacities[0]
        )
        """
        This updates the inventory levels of the distribution warehouses in a similar way to the factory.
        For each distribution warehouse j, the inventory level is updated by subtracting the products shipped from the warehouse (action.shipped_stocks[j])
        and adding the products received from the factory. The resulting inventory level is then capped at the maximum storage capacity of the warehouse (self.storage_capacities[j+1]).
        """
        for j in range(self.distr_warehouses_num):
            next_state.distr_warehouses_stocks[j] = np.minimum(
                np.subtract(np.add(state.distr_warehouses_stocks[j],
                                   action.shipped_stocks[j]),
                            demands[j]
                            ),
                self.storage_capacities[j+1]
            )

        # revenues
        total_revenues = np.dot(self.sale_prices,
                                np.sum(demands, axis=0))
        # production costs
        total_production_costs = np.dot(self.production_costs,
                                        action.production_level)
        # transportation costs
        total_transportation_costs = np.dot(
            self.transportation_costs.flatten(),
            action.shipped_stocks.flatten())
        # storage costs
        total_storage_costs = np.dot(
            self.storage_costs.flatten(),
            np.maximum(next_state.stock_levels(),
                       np.zeros(
                           ((self.distr_warehouses_num+1) *
                            self.product_types_num),
                           dtype=np.int32)
                       )
        )
        # penalty costs (minus sign because stock levels would be already
        # negative in case of unfulfilled demand)
        total_penalty_costs = -np.dot(
            self.penalty_costs,
            np.add(
                np.sum(
                    np.minimum(next_state.distr_warehouses_stocks,
                               np.zeros(
                                   (self.distr_warehouses_num,
                                    self.product_types_num),
                                   dtype=np.int32)
                               ),
                    axis=0),
                np.minimum(next_state.factory_stocks,
                           np.zeros(
                               (self.product_types_num,),
                               dtype=np.int32)
                           )
            )
        )
        # reward function
        reward = total_revenues - total_production_costs - \
            total_transportation_costs - total_storage_costs - \
            total_penalty_costs

        # the actual demand for the current time step will not be known until
        # the next time step. This implementation choice ensures that the agent
        # may benefit from learning the demand pattern so as to integrate a
        # sort of demand forecasting directly into the policy
        self.demand_history.append(demands)
        
        # actual time step value is not observed (for now)
        self.t += 1

        return next_state, reward, self.t == self.T-1

# Scratch implementation

Description of main elements:

__init__ method (initialization of agent)

reset method (startin point of the envinment)

step method (use so that the ou perform one step inside the environment)

observation_space attribute (inventories for each product for each warehouse + factory)

action_space attribute (production units of each product and shipping units of each product to each warehouse)

sale_prices attribute (sale price fo each product)

warehouse_storage_costs attribute (storage cost for each product in each warehouse)

manufacturer_storage_costs attribute (storage costs for each product in inventory)

manufacturer_storage_cap attribute (maximum storage capacity for each product for the factory)

warehouse_storage_capacities attribute (maximum storage capacity for each product for each warehouse)

production_costs attribute (production cost for each product)

production_capacity attribute (maximum production capacity for each product)

shipping_costs attribute (shipping costs for each product to each warehouse)

penalty_costs attribute (fleating cost of unsatisfied demand)

num_products attribute (number of products)

num_distr_warehouses attribute (number of warehouses)

truck_capacity attribute (size of deliverable batches for each product)

T attribute (terminal step)

manufacturer_inventory attribute (inventory in the factory for each product)

warehouse_inventories attribute (inventory for each product in each warehouse)

demands attribute (demand for that period for each product from each warehouse)

In [ ]:
from numpy.lib.shape_base import column_stack
import gym
from gym import spaces
import numpy as np

class SupplyChainEnv(): # before inside was written gym.Env
    def __init__(self, sale_prices, warehouse_storage_costs, manufacturer_storage_costs, manufacturer_storage_cap, warehouse_storage_capacities, production_costs, production_capacity, shipping_costs, demand_distribution, truck_capacity, num_products=2, num_distr_warehouses=2):

        ''' Define the supply chain parameters'''

        self.demand_distribution = demand_distribution
        self.sale_prices = sale_prices
        self.production_costs = production_costs
        self.manufacturer_storage_cap = manufacturer_storage_cap
        self.warehouse_storage_cap = warehouse_storage_capacities
        self.warehouse_storage_costs = warehouse_storage_costs
        self.manufacturer_storage_costs = manufacturer_storage_costs
        self.shipping_costs = shipping_costs
        self.penalty_costs = 1.5 * self.sale_prices #for unstatisfied demand, 1 for the fleating cost plus 0.5 to account for unsatisfaction of client (may not come back)
        self.production_capacity = production_capacity

        self.num_products = num_products
        self.num_distr_warehouses = num_distr_warehouses
        self.truck_capacity = truck_capacity
        # final time step (e.g., an episode takes 12 time steps, monthly decisions)
        self.T = 12
        self.t=0
        # Define the state space
        '''
        This Box space allows each element of the observation to take on any real value between 0 and infinity,
        representing the inventory levels of each product at each warehouse and the manufacturer.
        When you provide a single value low argument, it is interpreted as the lower limit for all dimensions of the space.
        However, when you provide an array-like object for low, the values are treated as the lower limits for each
        corresponding dimension of the space. Similarly, the same logic applies for the high argument, which represents the upper limits of the space.
        '''
        
        observation_high = np.concatenate([self.manufacturer_storage_cap, self.warehouse_storage_cap.flatten()])
        self.observation_space = spaces.Box(low=np.zeros_like(observation_high), high=observation_high, dtype=np.int32) #lowest inventory level is zero

        ''' Define the action space '''
        # Production quantities of each product at the manufacturer
        manufacturer_production_space = spaces.Box(low=0, high=self.production_capacity, shape=(num_products,), dtype=np.int32)
        # Amount of each product that will be sent from the manufacturer to each distribution warehouse, high will be the maximum a truck can carry
        warehouse_shipping_space = spaces.Box(low=0, high=np.concatenate([self.truck_capacity,self.truck_capacity]), shape=(num_products * num_distr_warehouses,), dtype=np.int32)

        # Over action space
        self.action_space = spaces.Tuple((manufacturer_production_space, warehouse_shipping_space))

        self.reset() # Reset the environment to its initial state at the beginning of a new episode (function defined just below)

    def reset(self):
        # Initialize the inventory levels and costs
        self.manufacturer_inventory = np.zeros(self.num_products)
        self.warehouse_inventories = np.zeros((self.num_distr_warehouses, self.num_products))
        
        # Generate initial demand for each warehouse
        self.generate_demand()
        print(self.demands)
        # t is current time step
        self.t = 0

        # Return the initial observation
        return self._get_observation()

    def step(self, action):

        manufacturer_production, warehouse_shipping = action

        #-------------UPDATING INVENTORIES------------------------
        # Update manufacturer inventory 
        self.manufacturer_inventory += manufacturer_production - np.sum(warehouse_shipping, axis=0)
        
        # Update warehouse inventories on shipped inventory '''CONSIDER IF FUTHER FIXING'''
        #basically if it chose to produce more products that what can be stored we set inventory to max capacity, so there will be a production cost of these products but not a revenue coming from them
        self.warehouse_inventories = np.minimum(self.warehouse_inventories + warehouse_shipping, self.warehouse_storage_cap)
        
        #-------------CALCULATING REVENUE----------------------------

        revenue = np.sum(self.sale_prices * np.sum(np.minimum(self.demands, self.warehouse_inventories))) # taking the minimum to account for the case we do not have enough to satisfy demand
        
        # Update warehouse inventories (after satisfaction of demand)
        self.warehouse_inventories -= self.demands # this will result in negative values for unsatisfied demand

        #-------------CALCULATING COSTS------------------------------

        # Calculate production costs
        production_costs = np.sum(manufacturer_production * self.production_costs)

        # Calculate shipping costs
        shipping_costs = np.sum(np.stack([self.truck_capacity,self.truck_capacity]) * self.shipping_costs)
        
        # Calculate penalty costs for unsatisfied demand
        unsatisfied_demand = np.minimum(self.warehouse_inventories, 0)
        penalty_costs = -np.sum(np.sum(unsatisfied_demand, axis=0) * self.penalty_costs) # minus sign because stock levels would be already negative in case of unfulfilled demand

        # Update warehouse inventories so that they do not have minus
        self.warehouse_inventories = np.maximum(self.warehouse_inventories, 0)

        # Calculate storage costs associated with the warehouse inventories, storage costs are for the inventory not yet sold
        warehouse_storage_costs = np.sum(self.warehouse_storage_costs * self.warehouse_inventories)        
        manufacturer_storage_costs = np.sum(self.manufacturer_storage_costs * self.manufacturer_inventory)     
        # Calculate total cost
        total_cost = shipping_costs + warehouse_storage_costs + manufacturer_storage_costs + production_costs + penalty_costs
        #print('The total cost is',total_cost)
        #-------------------CALCULATING GROSS REVENUE = reward----------------------
        reward = revenue - total_cost

        self.t += 1
        # Generate new demand for each warehouse
        self.generate_demand()
        print(self.demands)
        # Return the new observation, reward, done flag, and info dictionary
        observation = self._get_observation()
        done = False
        info = {}
        
        return observation, reward, done, info

    def generate_demand(self):
        self.demands = np.zeros((self.num_distr_warehouses, self.num_products))
        for prod in range(self.num_products):
          for ware in range(self.num_distr_warehouses):
            demand_dist = self.demand_distribution[prod][ware][self.t]
            self.demands[prod][ware] = np.random.choice(demand_dist)

    def _get_observation(self):
        # Concatenate the manufacturer inventory and warehouse inventories
        inventories = np.concatenate([self.manufacturer_inventory, self.warehouse_inventories.flatten()]) #.flatten returns a copy on an array collapsed into one dimension
        
        # Concatenate the inventories with the demands
        observation = np.concatenate([inventories, self.demands.flatten()])
        
        return observation 
        '''
        so returned value is a numpy array with:
        1st row : manufacturer_inventory (size : num_products)
        2nd row : warehouse_inventories (size : num_products * num_warehouses)
        3rd row : demands (size : num_products * num_warehouses)
        '''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# IDEA

we can create 2 implementations, the second one not having the demand upfront so that we can add a forecasting element and compare performances

Let's create a scenario to check if things work

In [ ]:
sale_prices = np.array([20,10])
warehouse_storage_costs = np.array([[5,10],[7,12]])
manufacturer_storage_costs = np.array([5,10])
manufacturer_storage_cap = np.array([200,250])
warehouse_storage_capacities = np.array([[200,250],[400,500]])
production_costs = np.array([10,5])
production_capacity = np.array([700,500])
shipping_costs = np.array([[10,12],[9,11]])
truck_capacity = np.array([100,100])
demand_distribution = [[[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]],[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]]],\
                       [[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]],[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]]]]
num_products = 2
num_distr_warehouses = 2

env = SupplyChainEnv(sale_prices, warehouse_storage_costs, manufacturer_storage_costs, manufacturer_storage_cap, warehouse_storage_capacities, production_costs, production_capacity, shipping_costs, demand_distribution, truck_capacity, num_products=2, num_distr_warehouses=2)

[[1. 1.]
 [1. 1.]]


In [ ]:
env.action_space

Tuple(Box(0, [700 500], (2,), int32), Box(0, 100, (4,), int32))

In [ ]:
env.observation_space

Box(0, [200 250 200 250 400 500], (6,), int32)

In [ ]:
observation = env.reset()
print(observation)

[[1. 1.]
 [1. 1.]]
[0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]


In [ ]:
action = (np.array([100, 100]), np.array([[10, 20], [30, 40]]))
next_observation, reward, done, info = env.step(action)
print(next_observation)
print(reward)

[[2. 2.]
 [2. 2.]]
[60. 40.  9. 19. 29. 39.  2.  2.  2.  2.]
-7186.0
